In [1]:
# Install PySpark on the Colab machine - code in "חומר עזר קולאב"
# Cell 1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
# Cell 2
!pip install --force-reinstall pyspark==3.4
!pip install findspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_412"
OpenJDK Runtime Environment (build 1.8.0_412-8u412-ga-1~22.04.1-b08)
OpenJDK 64-Bit Server VM (build 25.412-b08, mixed mode)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 25.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=226db525b3270bc707540597013df63d42e6dee81d865f44f0b71fee0baefb38
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import*
import os
import re


In [3]:
# Before getting/creating the Session, we can try to modify parameters.
spark = SparkSession.builder.appName('Spark - Lab 3')\
    .getOrCreate()
sc = spark.sparkContext
# keep only important logs
spark.sparkContext.setLogLevel("ERROR")

In [4]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [5]:
spark

In [47]:
with open("view_data_lab3.csv", 'r') as file:
    lines = file.readlines()

header = lines[0].strip().split('\t')
rows = [line.strip().split(',') for line in lines[1:]]
items = sc.parallelize(rows) \
          .filter(lambda item: 200000 <= int(item[0]) < 230000) \
          .map(lambda item: ((item[3], item[4]), 1)) \
          .reduceByKey(lambda a, b: a + b) \
          .map(lambda x: (x[0][0], (x[1], 1))) \
          .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
          .mapValues(lambda x: x[0] / x[1]) \
          .map(lambda x: (x[1], x)) \
          .sortByKey(False).map(lambda x: x[1]) \

for i, j in items.take(5):
  print(i, j)

7.5E+14 64.0
7.46E+14 8.0
7.503E+14 7.166666666666667
8.00001E+11 5.6
8.4843E+14 5.4
